In [1]:
import pandas as pd

In [2]:
#!pip install pyarrow

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [5]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
df = df_jan

In [7]:
# Q1
print(f'Columns in January: {len(df.columns)}')

Columns in January: 19


In [8]:
# Q2
df['duration'] = ((df.tpep_dropoff_datetime-df.tpep_pickup_datetime).dt.seconds/60)

df.duration.std()

41.62919110966266

In [9]:
# Q3

# remove outliers
print(((df.duration >=1) & (df.duration <=60)).mean())

df = df[(df.duration >=1) & (df.duration <=60)]

0.9812212604417814


In [10]:
# Q4 

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_654/502099879.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [11]:
train_dict= df[categorical].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

In [13]:
print('Number of features')
len(dv.feature_names_)

Number of features


515

In [14]:
X_train

<3009176x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018352 stored elements in Compressed Sparse Row format>

In [15]:
# Q5

target = 'duration'
y_train = df[target].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)

In [18]:
#sns.distplot(y_pred, label='prediction')
#sns.distplot(y_train, label='actual')

In [19]:
mean_squared_error(y_train, y_pred, squared=False)

7.64926111619175

In [20]:
# Q6

# validation

df_feb['duration'] = ((df_feb.tpep_dropoff_datetime-df_feb.tpep_pickup_datetime).dt.seconds/60)

df_val = df_feb
df_val = df_val[(df_val.duration >=1) & (df_val.duration <=60)]

df_val[categorical] = df_val[categorical].astype(str)

transform_dict_val = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(transform_dict_val)

#y_pred_feb = lr.predict(X_feb)
#mean_squared_error(y_pred_feb, df_feb[target], squared=False)

/tmp/ipykernel_654/3305592655.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[categorical] = df_val[categorical].astype(str)


In [21]:
y_pred_val = lr.predict(X_val)

In [22]:
y_val = df_val[target].values
mean_squared_error(y_val, y_pred_val, squared=False)

7.8118197398327585